In [ ]:
!pip install objectnat
!pip install pyarrow 
!pip install IduEdu --upgrade

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pickle
import os
import networkx as nx

from decimal import *
from objectnat import get_service_provision
from objectnat import recalculate_links
from objectnat import clip_provision

In [ ]:
# Определяем путь до папки с файлами

data_path = 'F:/Резерв/Учеба/НИР-3/Чат gpt/Исходные данные'

In [ ]:
# Загружаем заранее обработанный файл со зданиями

buildings = gpd.read_file(os.path.join(data_path, 'buildings.geojson'))

In [ ]:
# Проверяем, что файл со зданиями не содержит ошибок

buildings.info()

In [ ]:
# Определяем спрос для каждого диапазона времени. Коэффициенты 0.43 и 0.38 взяты из результатов социологического опроса

buildings['demand_30'] = (buildings['demand'] * 0.43).round().astype(int)
buildings['demand_60'] = (buildings['demand'] * 0.38).round().astype(int)
buildings['demand_90'] = buildings['demand'] - buildings['demand_30'] - buildings['demand_60']
buildings['total_demand'] = buildings['demand_30'] + buildings['demand_60'] + buildings['demand_90']

buildings

In [ ]:
# Загружаем заранее обработанный файл со скейт-парками

skateparks = gpd.read_file(os.path.join(data_path, 'skateparks.geojson'))

# Смотрим как он выглядит и проверяем, что все атрибуты на месте

skateparks.explore(column='capacity',tiles='CartoDB positron')

In [ ]:
# Вычисление суммарного спроса по категориям

total_population = buildings['demand'].sum()
print(f"Суммарная численность населения: {total_population}")

demand_30 = buildings['demand_30'].sum()
print(f"Суммарный спрос в границах 30 минут: {demand_30}")

demand_60 = buildings['demand_60'].sum()
print(f"Суммарный спрос в границах 60 минут: {demand_60}")

demand_90 = buildings['demand_90'].sum()
print(f"Суммарный спрос в границах 90 минут: {demand_90}")

total_demand = buildings['total_demand'].sum()
print(f"Общий суммарный спрос: {total_demand}")

total_capacity = skateparks['capacity'].sum()
print("Суммарное количество людей, которые может обслужить скейт-парк:", total_capacity)

In [ ]:
# Загрузка матриц смежности

matrix = pd.read_csv('F:/Резерв/Учеба/НИР-3/Чат gpt/matrix_time.csv', index_col=0)
matrix_30 = pd.read_csv('F:/Резерв/Учеба/НИР-3/Чат gpt/matrix_30.csv', index_col=0)
matrix_60 = pd.read_csv('F:/Резерв/Учеба/НИР-3/Чат gpt/matrix_60.csv', index_col=0)
matrix_90 = pd.read_csv('F:/Резерв/Учеба/НИР-3/Чат gpt/matrix_90.csv', index_col=0)

In [ ]:
class CapacityKeyError(KeyError):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return "CapacityKeyError, {0} ".format(self.message)

        return (
            "Column 'capacity' was not found in provided 'services' GeoDataFrame. This attribute "
            "corresponds to the total capacity for each service."
        )


class CapacityValueError(ValueError):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return "CapacityValueError, {0} ".format(self.message)

        return "Column 'capacity' in 'services' GeoDataFrame  has no valid value."


class DemandKeyError(KeyError):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return "DemandKeyError, {0} ".format(self.message)

        return (
            "The column 'demand' was not found in the provided 'demanded_buildings' GeoDataFrame. "
            "This attribute corresponds to the number of demands for the selected service in each building."
        )


class DemandValueError(ValueError):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return "DemandValueError, {0} ".format(self.message)
        return "Column 'demand' in 'demanded_buildings' GeoDataFrame  has no valid value."


In [ ]:
# pylint: disable=singleton-comparison
from typing import Tuple

import geopandas as gpd
import numpy as np
import pandas as pd
from pandarallel import pandarallel
from shapely import LineString

from objectnat import config

from provision_exceptions import CapacityKeyError, DemandKeyError

logger = config.logger


class Provision:
    """
    Represents the logic for city provision calculations using a gravity or linear model.

    Args:
        services (InstanceOf[gpd.GeoDataFrame]): GeoDataFrame representing the services available in the city.
        demanded_buildings (InstanceOf[gpd.GeoDataFrame]): GeoDataFrame representing the buildings with demands for services.
        adjacency_matrix (InstanceOf[pd.DataFrame]): DataFrame representing the adjacency matrix between buildings.
        threshold (int): Threshold value for the provision calculations.
        calculation_type (str, optional): Type of calculation ("gravity" or "linear"). Defaults to "gravity".

    Returns:
        Provision: The CityProvision object.

    Raises: KeyError: If the 'demand' column is missing in the provided 'demanded_buildings' GeoDataFrame,
    or if the 'capacity' column is missing in the provided 'services' GeoDataFrame. ValueError: If the 'capacity'
    column in 'services' or 'demand' column  'demanded_buildings' GeoDataFrame has no valid value.
    """

    destination_matrix = None

    def __init__(
        self,
        services: gpd.GeoDataFrame,
        demanded_buildings: gpd.GeoDataFrame,
        adjacency_matrix: pd.DataFrame,
        threshold: int,
    ):
        self.services = self.ensure_services(services.copy())
        self.demanded_buildings = self.ensure_buildings(demanded_buildings.copy())
        self.adjacency_matrix = self.delete_useless_matrix_rows_columns(
            adjacency_matrix.copy(), demanded_buildings, services
        ).copy()
        self.threshold = threshold
        self.check_crs(self.demanded_buildings, self.services)
        pandarallel.initialize(progress_bar=False, verbose=0, use_memory_fs=config.pandarallel_use_file_system)

    @staticmethod
    def ensure_buildings(v: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
        if "demand" not in v.columns:
            raise DemandKeyError
        v["demand_left"] = v["demand"]
        return v

    @staticmethod
    def ensure_services(v: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
        if "capacity" not in v.columns:
            raise CapacityKeyError
        v["capacity_left"] = v["capacity"]
        return v

    @staticmethod
    def check_crs(demanded_buildings, services):
        assert (
            demanded_buildings.crs == services.crs
        ), f"\nThe CRS in the provided geodataframes are different.\nBuildings CRS:{demanded_buildings.crs}\nServices CRS:{services.crs} \n"

    @staticmethod
    def delete_useless_matrix_rows_columns(adjacency_matrix, demanded_buildings, services):
        adjacency_matrix.index = adjacency_matrix.index.astype(int)

        builds_indexes = set(demanded_buildings.index.astype(int).tolist())
        rows = set(adjacency_matrix.index.astype(int).tolist())
        dif = rows ^ builds_indexes
        adjacency_matrix.drop(index=(list(dif)), axis=0, inplace=True)

        service_indexes = set(services.index.astype(int).tolist())
        columns = set(adjacency_matrix.columns.astype(int).tolist())
        dif = columns ^ service_indexes
        adjacency_matrix.drop(columns=(list(dif)), axis=0, inplace=True)
        return adjacency_matrix.transpose()

    def run(self) -> Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame]:

        def apply_function_based_on_size(df, func, axis, threshold=100):
            if len(df) > threshold:
                return df.parallel_apply(func, axis=axis)
            return df.apply(func, axis=axis)

        def calculate_flows_y(loc):
            import numpy as np  # pylint: disable=redefined-outer-name,reimported,import-outside-toplevel
            import pandas as pd  # pylint: disable=redefined-outer-name,reimported,import-outside-toplevel

            c = services_table.loc[loc.name]["capacity_left"]
            a = services_table.loc[loc.name]["Rating"]
            p = a / loc / loc
            p = p / p.sum()
            threshold = p.quantile(best_houses)
            p = p[p >= threshold]
            p = p / p.sum()
            if p.sum() == 0:
                return loc
            rng = np.random.default_rng(seed=0)
            r = pd.Series(0, p.index)
            choice = np.unique(rng.choice(p.index, int(c), p=p.values), return_counts=True)
            choice = r.add(pd.Series(choice[1], choice[0]), fill_value=0)

            return choice

        def balance_flows_to_demands(loc):
            import numpy as np  # pylint: disable=redefined-outer-name,reimported,import-outside-toplevel
            import pandas as pd  # pylint: disable=redefined-outer-name,reimported,import-outside-toplevel

            d = houses_table.loc[loc.name]["demand_left"]
            loc = loc[loc > 0]
            if loc.sum() > 0:
                p = loc / loc.sum()
                rng = np.random.default_rng(seed=0)
                r = pd.Series(0, p.index)
                choice = np.unique(rng.choice(p.index, int(d), p=p.values), return_counts=True)
                choice = r.add(pd.Series(choice[1], choice[0]), fill_value=0)
                choice = pd.Series(
                    data=np.minimum(loc.sort_index().values, choice.sort_index().values),
                    index=loc.sort_index().index,
                )
                return choice
            return loc

        logger.debug(
            f"Calculating provision from {len(self.services)} services to {len(self.demanded_buildings)} buildings."
        )

        distance_matrix = self.adjacency_matrix
        destination_matrix = pd.DataFrame(
            0,
            index=distance_matrix.index,
            columns=distance_matrix.columns,
            dtype=int,
        )
        distance_matrix = distance_matrix.where(distance_matrix <= self.threshold * 3, np.inf)

        houses_table = self.demanded_buildings[["demand", "demand_left"]].copy()
        services_table = self.services[["capacity", "capacity_left"]].copy()
        distance_matrix = distance_matrix.drop(
            index=services_table[services_table["capacity_left"] == 0].index.values,
            columns=houses_table[houses_table["demand_left"] == 0].index.values,
            errors="ignore",
        )
        distance_matrix = distance_matrix.loc[~(distance_matrix == np.inf).all(axis=1)]
        distance_matrix = distance_matrix.loc[:, ~(distance_matrix == np.inf).all(axis=0)]

        distance_matrix = distance_matrix + 1
        selection_range = (self.threshold + 1) / 2
        best_houses = 0.9
        while len(distance_matrix.columns) > 0 and len(distance_matrix.index) > 0:
            objects_n = sum(distance_matrix.shape)
            logger.debug(
                f"Matrix shape: {distance_matrix.shape},"
                f" Total objects: {objects_n},"
                f" Selection range: {selection_range},"
                f" Best houses: {best_houses}"
            )

            temp_destination_matrix = apply_function_based_on_size(
                distance_matrix, lambda x: calculate_flows_y(x[x <= selection_range]), 1
            )

            temp_destination_matrix = temp_destination_matrix.fillna(0)
            temp_destination_matrix = apply_function_based_on_size(temp_destination_matrix, balance_flows_to_demands, 0)
            temp_destination_matrix = temp_destination_matrix.fillna(0)
            temp_destination_matrix_aligned = temp_destination_matrix.reindex(
                index=destination_matrix.index, columns=destination_matrix.columns, fill_value=0
            )
            del temp_destination_matrix
            destination_matrix_np = destination_matrix.to_numpy()
            temp_destination_matrix_np = temp_destination_matrix_aligned.to_numpy()
            del temp_destination_matrix_aligned
            destination_matrix = pd.DataFrame(
                destination_matrix_np + temp_destination_matrix_np,
                index=destination_matrix.index,
                columns=destination_matrix.columns,
            )
            del destination_matrix_np, temp_destination_matrix_np
            axis_1 = destination_matrix.sum(axis=1).astype(int)
            axis_0 = destination_matrix.sum(axis=0).astype(int)

            services_table["capacity_left"] = services_table["capacity"].subtract(axis_1, fill_value=0)
            houses_table["demand_left"] = houses_table["demand"].subtract(axis_0, fill_value=0)
            del axis_1, axis_0
            distance_matrix = distance_matrix.drop(
                index=services_table[services_table["capacity_left"] == 0].index.values,
                columns=houses_table[houses_table["demand_left"] == 0].index.values,
                errors="ignore",
            )
            distance_matrix = distance_matrix.loc[~(distance_matrix == np.inf).all(axis=1)]
            distance_matrix = distance_matrix.loc[:, ~(distance_matrix == np.inf).all(axis=0)]

            selection_range *= 1.5
            if best_houses <= 0.1:
                best_houses = 0
            else:
                objects_n_new = sum(distance_matrix.shape)
                best_houses = objects_n_new / (objects_n / best_houses)

        logger.debug("Done!")
        del distance_matrix, houses_table, services_table
        self.destination_matrix = destination_matrix

        _additional_options(
            self.demanded_buildings,
            self.services,
            self.adjacency_matrix,
            self.destination_matrix,
            self.threshold,
        )

        return (
            self.demanded_buildings,
            self.services,
            _calc_links(
                self.destination_matrix,
                self.services,
                self.demanded_buildings,
                self.adjacency_matrix,
            ),
        )


def _calc_links(
    destination_matrix: pd.DataFrame,
    services: gpd.GeoDataFrame,
    buildings: gpd.GeoDataFrame,
    distance_matrix: pd.DataFrame,
):
    buildings_ = buildings.copy()
    services_ = services.copy()
    buildings_.geometry = buildings_.representative_point()
    services_.geometry = services_.representative_point()

    def subfunc(loc):
        try:
            return [
                {
                    "building_index": int(k),
                    "demand": int(v),
                    "service_index": int(loc.name),
                }
                for k, v in loc.to_dict().items()
            ]
        except:
            return np.NaN

    def subfunc_geom(loc):
        return LineString(
            (
                buildings_.geometry[loc["building_index"]],
                services_.geometry[loc["service_index"]],
            )
        )

    flat_matrix = destination_matrix.transpose().apply(lambda x: subfunc(x[x > 0]), result_type="reduce")

    distribution_links = gpd.GeoDataFrame(data=[item for sublist in list(flat_matrix) for item in sublist])

    distribution_links["distance"] = distribution_links.apply(
        lambda x: distance_matrix.loc[x["service_index"]][x["building_index"]],
        axis=1,
        result_type="reduce",
    )

    sel = distribution_links["building_index"].isin(buildings_.index.values) & distribution_links["service_index"].isin(
        services_.index.values
    )
    sel = distribution_links.loc[sel[sel].index.values]
    distribution_links = distribution_links.set_geometry(sel.apply(subfunc_geom, axis=1)).set_crs(buildings_.crs)
    distribution_links["distance"] = distribution_links["distance"].astype(float).round(2)
    return distribution_links


def _additional_options(
    buildings,
    services,
    matrix,
    destination_matrix,
    normative_distance,
):
    buildings["avg_dist"] = 0
    buildings["supplyed_demands_within"] = 0
    buildings["supplyed_demands_without"] = 0
    services["carried_capacity_within"] = 0
    services["carried_capacity_without"] = 0
    for i, loc in destination_matrix.iterrows():
        distances_all = matrix.loc[loc.name]
        distances = distances_all[distances_all <= normative_distance]
        s = matrix.loc[loc.name] <= normative_distance
        within = loc[s]
        without = loc[~s]
        within = within[within > 0]
        without = without[without > 0]
        buildings["avg_dist"] = (
            buildings["avg_dist"]
            .add(distances.multiply(within, fill_value=0), fill_value=0)
            .add(distances_all.multiply(without, fill_value=0), fill_value=0)
        )
        buildings["demand_left"] = buildings["demand_left"].sub(within.add(without, fill_value=0), fill_value=0)
        buildings["supplyed_demands_within"] = buildings["supplyed_demands_within"].add(within, fill_value=0)
        buildings["supplyed_demands_without"] = buildings["supplyed_demands_without"].add(without, fill_value=0)

        services.at[loc.name, "capacity_left"] = (
            services.at[loc.name, "capacity_left"] - within.add(without, fill_value=0).sum()
        )
        services.at[loc.name, "carried_capacity_within"] = (
            services.at[loc.name, "carried_capacity_within"] + within.sum()
        )
        services.at[loc.name, "carried_capacity_without"] = (
            services.at[loc.name, "carried_capacity_without"] + without.sum()
        )
    buildings["min_dist"] = matrix.min(axis=0).replace(np.inf, None)
    buildings["avg_dist"] = (buildings["avg_dist"] / (buildings["demand"] - buildings["demand_left"])).astype(
        np.float32
    )
    buildings["avg_dist"] = buildings.apply(
        lambda x: np.nan if (x["demand"] == x["demand_left"]) else round(x["avg_dist"], 2), axis=1
    )
    buildings["provison_value"] = (buildings["supplyed_demands_within"] / buildings["demand"]).astype(float).round(2)
    services["service_load"] = (services["capacity"] - services["capacity_left"]).astype(np.uint16)
    buildings["supplyed_demands_within"] = buildings["supplyed_demands_within"].astype(np.uint16)
    buildings["supplyed_demands_without"] = buildings["supplyed_demands_without"].astype(np.uint16)
    services["carried_capacity_within"] = services["carried_capacity_within"].astype(np.uint16)
    services["carried_capacity_without"] = services["carried_capacity_without"].astype(np.uint16)
    logger.debug("Done adding additional options")

In [ ]:
def get_service_provision(
    buildings: gpd.GeoDataFrame,
    adjacency_matrix: pd.DataFrame,
    services: gpd.GeoDataFrame,
    threshold: int,
    buildings_demand_column: str = "demand",
    services_capacity_column: str = "capacity",
    services_rating_column: str = "Rating",
) -> Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """Calculate load from buildings with demands on the given services using the distances matrix between them.

    Args:
        services (gpd.GeoDataFrame): GeoDataFrame of services
        adjacency_matrix (pd.DataFrame): DataFrame representing the adjacency matrix
        buildings (gpd.GeoDataFrame): GeoDataFrame of demanded buildings
        threshold (int): Threshold value
        buildings_demand_column (str): column name of buildings demands
        services_capacity_column (str): column name of services capacity
    Returns:
        Tuple[gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame]: Tuple of GeoDataFrames representing provision
        buildings, provision services, and provision links
    """
    buildings = buildings.copy()
    services = services.copy()
    adjacency_matrix = adjacency_matrix.copy()
    buildings["demand"] = buildings[buildings_demand_column]
    services["capacity"] = services[services_capacity_column]
    services["Rating"] = services[services_rating_column]


    provision_buildings, provision_services, provision_links = Provision(
        services=services,
        demanded_buildings=buildings,
        adjacency_matrix=adjacency_matrix,
        threshold=threshold,
    ).run()
    return provision_buildings, provision_services, provision_links

In [ ]:
# Расчет оценки обеспеченности

from objectnat import get_service_provision

buildings['demand'] = buildings['demand_30'] # для сценария 1 указывается 'demand_30', для сценария 2 'demand_60', для сценария 3 'demand_90'
#skateparks['capacity'] = buildings['total_demand'].sum() # для экспериментального сценария
builds = buildings.to_crs(32636)
services = skateparks.to_crs(32636)
adjacency_matrix = matrix_30 # для сценария 1 указывается matrix_30, для сценария 2 matrix_60, для сценария 3 matrix_90
# Reading building and service data, reprojecting them to EPSG:32636 (UTM zone), and loading the precomputed adjacency matrix

adjacency_matrix.index = adjacency_matrix.index.astype(int)
services.index = services.index.astype(int)
builds.index = builds.index.astype(int)
adjacency_matrix.columns = adjacency_matrix.columns.astype(int)
# Ensuring the indices and columns of the adjacency matrix and GeoDataFrames are integers for proper matching

build_prov, services_prov, links_prov = get_service_provision(
    buildings=builds,
    services=services,
    adjacency_matrix=adjacency_matrix,
    threshold=30, # для сценария 1 threshold=30, для сценария 2 threshold=60, для сценария 3 threshold=90
)
# Calculating service provision for each building. The 'threshold' parameter (10) defines the maximum allowable distance (or time)
# beyond which buildings are considered to not meet the required service provision.

services_prov['load'] = services_prov['service_load'] / services_prov['capacity']

# Скачивание файлов для основных сценариев
build_prov.to_file(os.path.join(data_path,"build_prov_30.geojson"))
services_prov.to_file(os.path.join(data_path,"services_prov_30.geojson"))
links_prov.to_file(os.path.join(data_path,"links_prov_30.geojson"))

# Скачивание файлов для экспериментальных сценариев
#build_prov.to_file(os.path.join(data_path,"build_prov_exp_30.geojson"))
#services_prov.to_file(os.path.join(data_path,"services_prov_exp_30.geojson"))
#links_prov.to_file(os.path.join(data_path,"links_prov_exp_30.geojson"))

In [ ]:
# Визуализация полученных данных

# Visualize data
m1 = build_prov.reset_index().explore(column='avg_dist', cmap='RdYlGn_r', tiles='CartoDB positron')
# Visualizing buildings provision on a map using 'explore'. 
# The 'avg_dist' column (average distance to services) is visualized with a color map 'RdYlGn_r'

links_prov.explore(m=m1, column='service_index', cmap='prism', style_kwds={'opacity': 0.5})
# Visualizing links (connections between buildings and services) on the same map.
# 'service_index' shows which service each link corresponds to.

services_prov.explore(m=m1, color='red')
# Visualizing service locations on the same map, using red markers for easy distinction.

In [ ]:
# Сохранение полученных данных в файл

m1.save('Основной сценарий - 30 минут.html')
# m1.save('Экспериментальный сценарий - 30 минут.html')

Далее будет происходить агрегация полученых результатов по районам и подсчет итогового показателя обеспеченности

In [ ]:
# Загружаем нужный файл с посчитанной обеспеченностью для жилых домов и скейт-парков

build_prov = gpd.read_file(os.path.join(data_path, 'build_prov_30.geojson'))
services_prov = gpd.read_file(os.path.join(data_path, 'services_prov_30.geojson'))

# build_prov = gpd.read_file(os.path.join(data_path, 'build_prov_exp_90.geojson'))
# services_prov = gpd.read_file(os.path.join(data_path, 'services_prov_exp_90.geojson'))

# Загружаем слои с границами административных районов и муниципальных образований

administrative_blocks = gpd.read_file(os.path.join(data_path, 'administrative_blocks.geojson'))
munitipal_blocks = gpd.read_file(os.path.join(data_path, 'munitipal_blocks.geojson'))

In [ ]:
# Агрегация данных по зданиям по административным районам

aggregated = build_prov.groupby('Administrative').agg(
    total_demand=('total_demand', 'sum'),   # Вычисление суммарного спроса
    supplyed_demands_within=('supplyed_demands_within', 'sum'), # Вычисление суммарного спроса, удовлетворенного в границах доступности
    supplyed_demands_without=('supplyed_demands_without', 'sum'),   # Вычисление суммарного спроса, удовлетворенного вне границ доступности
    total_left_demand=('demand_left', 'sum'),    # Вычисление суммарного спроса, оставшегося неудовлетворенным
    mean_dist=('avg_dist','mean'),  # Вычисление среднего расстояния до скейт-парка
    mean_prov=('provison_value','mean')    # Вычисление среднего значения обеспеченности
).reset_index()

aggregated['total_left_demand'] = aggregated['total_left_demand'].astype('int32')
aggregated['mean_dist'] = aggregated['mean_dist'].round(2)
aggregated['mean_prov'] = aggregated['mean_prov'].round(3)
aggregated['level_of_demand'] = round(((aggregated['total_demand'] - aggregated['total_left_demand']) / aggregated['total_demand']),3)

# Агрегация данных по скейт-паркам по административным районам

skateparks_aggregated = services_prov.groupby('Administrative').agg(
    skateparks_count=('TableID','count'), # Вычисление общего количества скейт-парков
    sum_of_capacity=('capacity', 'sum'), # Вычисление суммарной вместимости скейт-парков
    carried_capacity_within=('carried_capacity_within', 'sum'), # Вычисление суммарной вместимости, используемой в границах доступности
    carried_capacity_without=('carried_capacity_without', 'sum'), # Вычисление суммарной вместимости, используемой вне границ доступности
    capacity_left=('capacity_left', 'sum'), # Вычисление суммарной вестимости оставшейся неиспользуемой вместимости
    service_load=('service_load', 'mean'), # Вычисление среднего количества посетителей скейт-парка
    mean_load=('load', 'mean') # Вычисление среднего уровня загруженности скейт-парка
).reset_index()

skateparks_aggregated['skateparks_count'] = skateparks_aggregated['skateparks_count'].astype('int32')
skateparks_aggregated['service_load'] = skateparks_aggregated['service_load'].round(0).astype('int32')
skateparks_aggregated['mean_load'] = skateparks_aggregated['mean_load'].round(3)

# Объединение данных по домам и скейт-паркам

merged_df = pd.merge(aggregated, skateparks_aggregated, on='Administrative', how='left')

# Удаление NaN-значений и приведение к нужному типу данных
merged_df['skateparks_count'] = merged_df['skateparks_count'].fillna(0).astype('int32')
merged_df['sum_of_capacity'] = merged_df['sum_of_capacity'].fillna(0).astype('int32')
merged_df['carried_capacity_within'] = merged_df['carried_capacity_within'].fillna(0).astype('int32')
merged_df['carried_capacity_without'] = merged_df['carried_capacity_without'].fillna(0).astype('int32')
merged_df['capacity_left'] = merged_df['capacity_left'].fillna(0).astype('int32')
merged_df['service_load'] = merged_df['service_load'].fillna(0).astype('int32')
merged_df['mean_load'] = merged_df['mean_load'].fillna(0)

merged_df

In [ ]:
merged_df.info()

In [ ]:
# Определение функции для линейной нормализации
def min_max_normalization(column):
    return (column - column.min()) / (column.max() - column.min())

# Определение функции для обратной нормализации
def inverse_min_max_normalization(column):
    return 1 - (column - column.min()) / (column.max() - column.min())

# Определяем суммарный уровень загруженности скейт-парков
merged_df['skateparks_load'] = merged_df['skateparks_count'] * merged_df['mean_load']

# Применение функции к нужным столбцам
normalized_layer = merged_df.copy()  # Создаем копию DataFrame для нормализации
normalized_layer[['norm_mean_provision', 'norm_skateparks_load_level']] = normalized_layer[['mean_prov', 'skateparks_load']].apply(min_max_normalization)
normalized_layer[['norm_mean_dist']] = normalized_layer[['mean_dist']].apply(inverse_min_max_normalization)

# Расчет итоговой оценки
normalized_layer['final_assessment'] = normalized_layer['norm_mean_dist'] + normalized_layer['norm_mean_provision'] + normalized_layer['norm_skateparks_load_level']

# Округление полученных значений
normalized_layer[['norm_mean_provision', 'norm_skateparks_load_level', 'norm_mean_dist', 'final_assessment']] = normalized_layer[['norm_mean_provision', 'norm_skateparks_load_level', 'norm_mean_dist', 'final_assessment']].round(3)

# Визуализация данных
normalized_layer

In [ ]:
# Объединяем агрегированные данные с геометрией административных районов и сохраняем в файл
merged_gdf = administrative_blocks.merge(normalized_layer, on='Administrative')
merged_gdf.to_file(os.path.join(data_path,"Основной сценарий 30 минут - Административные районы.geojson"))
# merged_gdf.to_file(os.path.join(data_path,"Экспериментальный сценарий 30 минут - Административные районы.geojson"))

# Объединяем агрегированные данные с геометрией административных районов и сохраняем в файл
map = merged_gdf.explore(column='final_assessment', cmap='RdYlGn', tiles='CartoDB positron')
map.save('Основной сценарий 30 минут - Административные районы.html')
# map.save('Экспериментальный сценарий 30 минут - Административные районы.html')
map

Повторяем все тоже самое, но уже не для административных районов, а для муниципальных образований

In [ ]:
# Агрегация данных по зданиям по муниципальным образованиям

mun_aggregated = build_prov.groupby('Munitipal').agg(
    total_demand=('total_demand', 'sum'),   # Вычисление суммарного спроса
    supplyed_demands_within=('supplyed_demands_within', 'sum'), # Вычисление суммарного спроса, удовлетворенного в границах доступности
    supplyed_demands_without=('supplyed_demands_without', 'sum'),   # Вычисление суммарного спроса, удовлетворенного вне границ доступности
    total_left_demand=('demand_left', 'sum'),    # Вычисление суммарного спроса, оставшегося неудовлетворенным
    mean_dist=('avg_dist','mean'),  # Вычисление среднего расстояния до скейт-парка
    mean_prov=('provison_value','mean')    # Вычисление среднего значения обеспеченности
).reset_index()

mun_aggregated['total_left_demand'] = mun_aggregated['total_left_demand'].astype('int32')
mun_aggregated['mean_dist'] = mun_aggregated['mean_dist'].round(2).fillna(0)
mun_aggregated['mean_prov'] = mun_aggregated['mean_prov'].round(3).fillna(0)
mun_aggregated['level_of_demand'] = round(((mun_aggregated['total_demand'] - mun_aggregated['total_left_demand']) / mun_aggregated['total_demand']),3).fillna(0)

# Агрегация данных по скейт-паркам по административным районам

skateparks_aggregated = services_prov.groupby('Munitipal').agg(
    skateparks_count=('TableID','count'), # Вычисление общего количества скейт-парков
    sum_of_capacity=('capacity', 'sum'), # Вычисление суммарной вместимости скейт-парков
    carried_capacity_within=('carried_capacity_within', 'sum'), # Вычисление суммарной вместимости, используемой в границах доступности
    carried_capacity_without=('carried_capacity_without', 'sum'), # Вычисление суммарной вместимости, используемой вне границ доступности
    capacity_left=('capacity_left', 'sum'), # Вычисление суммарной вестимости оставшейся неиспользуемой вместимости
    service_load=('service_load', 'mean'), # Вычисление среднего количества посетителей скейт-парка
    mean_load=('load', 'mean') # Вычисление среднего уровня загруженности скейт-парка
).reset_index()

skateparks_aggregated['skateparks_count'] = skateparks_aggregated['skateparks_count'].astype('int32')
skateparks_aggregated['service_load'] = skateparks_aggregated['service_load'].round(0).astype('int32')
skateparks_aggregated['mean_load'] = skateparks_aggregated['mean_load'].round(3)

# Объединение данных по домам и скейт-паркам

mun_merged_df = pd.merge(mun_aggregated, skateparks_aggregated, on='Munitipal', how='left')

# Удаление NaN-значений и приведение к нужному типу данных
mun_merged_df['skateparks_count'] = mun_merged_df['skateparks_count'].fillna(0).astype('int32')
mun_merged_df['sum_of_capacity'] = mun_merged_df['sum_of_capacity'].fillna(0).astype('int32')
mun_merged_df['carried_capacity_within'] = mun_merged_df['carried_capacity_within'].fillna(0).astype('int32')
mun_merged_df['carried_capacity_without'] = mun_merged_df['carried_capacity_without'].fillna(0).astype('int32')
mun_merged_df['capacity_left'] = mun_merged_df['capacity_left'].fillna(0).astype('int32')
mun_merged_df['service_load'] = mun_merged_df['service_load'].fillna(0).astype('int32')
mun_merged_df['mean_load'] = mun_merged_df['mean_load'].fillna(0)

mun_merged_df

In [ ]:
mun_merged_df.info()

In [ ]:
# Определение функции для линейной нормализации
def min_max_normalization(column):
    return (column - column.min()) / (column.max() - column.min())

# Определение функции для обратной нормализации
def inverse_min_max_normalization(column):
    return 1 - (column - column.min()) / (column.max() - column.min())

# Определяем суммарный уровень загруженности скейт-парков
mun_merged_df['skateparks_load'] = mun_merged_df['skateparks_count'] * mun_merged_df['mean_load']

# Применение функции к нужным столбцам
normalized_layer = mun_merged_df.copy()  # Создаем копию DataFrame для нормализации
normalized_layer[['norm_mean_provision', 'norm_skateparks_load_level']] = normalized_layer[['mean_prov', 'skateparks_load']].apply(min_max_normalization)
normalized_layer[['norm_mean_dist']] = normalized_layer[['mean_dist']].apply(inverse_min_max_normalization)

# Расчет итоговой оценки
normalized_layer['final_assessment'] = normalized_layer['norm_mean_dist'] + normalized_layer['norm_mean_provision'] + normalized_layer['norm_skateparks_load_level']

# Округление полученных значений
normalized_layer[['norm_mean_provision', 'norm_skateparks_load_level', 'norm_mean_dist', 'final_assessment']] = normalized_layer[['norm_mean_provision', 'norm_skateparks_load_level', 'norm_mean_dist', 'final_assessment']].round(3)

# Визуализация данных
normalized_layer

In [ ]:
# Объединяем агрегированные данные с геометрией административных районов и сохраняем в файл

mun_merged_gdf = munitipal_blocks.merge(normalized_layer, on='Munitipal')
mun_merged_gdf.to_file(os.path.join(data_path,"Основной сценарий 30 минут - Муниципальные образования.geojson"))
# mun_merged_gdf.to_file(os.path.join(data_path,"Экспериментальный сценарий 30 минут - Муниципальные образования.geojson"))

# Объединяем агрегированные данные с геометрией административных районов и сохраняем в файл

map = mun_merged_gdf.explore(column='final_assessment', cmap='RdYlGn', tiles='CartoDB positron')
map.save('Основной сценарий 30 минут - Муниципальные образования.html')
# map.save('Экспериментальный сценарий 30 минут - Муниципальные образования.html')
map

In [ ]:
# Работа со слоем зданий в рамках одного района
# Выбираем из датафрейма с обеспеченностью ЗДАНИЙ только один район и создаем новый датафрейм

adm_name = 'Приморский'
mun_name = 'муниципальный округ № 65'

build_prov = gpd.read_file(os.path.join(data_path, 'build_prov_30.geojson'))
district_build_prov = build_prov[build_prov['Administrative'] == adm_name]
#district_build_prov = build_prov[build_prov['Munitipal'] == mun_name]

# Вычисление суммарного спроса
total_demand = district_build_prov['demand'].sum()
print(f"Суммарный спрос: {total_demand}")

# Вычисление суммарного спроса, удовлетворенного в границах доступности
supplyed_demands_within = district_build_prov['supplyed_demands_within'].sum()
print(f"Суммарный спрос, удовлетворенный в границах доступности: {supplyed_demands_within}")

# Вычисление суммарного спроса, удовлетворенного вне границ доступности
supplyed_demands_without = district_build_prov['supplyed_demands_without'].sum()
print(f"Суммарный спрос, удовлетворенный вне границ доступности: {supplyed_demands_without}")

# Вычисление суммарного спроса, оставшегося неудовлетворенным
total_left_demand = district_build_prov['demand_left'].sum()
print(f"Суммарный спрос, оставшийся неудовлетворенным: {total_left_demand}")

# Вычисление уровня удовлетворения спроса
level_demand = round(((district_build_prov['demand'].sum() - district_build_prov['demand_left'].sum()) / district_build_prov['demand'].sum()),3)
print(f"Уровень удовлетворения спроса: {level_demand}")

# Вычисление среднего расстояния до скейт-парка
mean_dist = round((district_build_prov['avg_dist'].mean()),2)
print(f"Среднее значение расстояния до скейт-парка: {mean_dist}")

# Вычисление среднего значения обеспеченности
mean_prov = round((district_build_prov['provison_value'].mean()),3)
print(f"Среднее значение обеспеченности скейт-парками: {mean_prov}")

In [ ]:
# Работа со слоем сервисов 
# Выбираем из датафрейма с обеспеченностью СКЕЙТ-ПАРКОВ только один район и создаем новый датафрейм

adm_name = 'Приморский'
mun_name = 'муниципальный округ № 65'

services_prov = gpd.read_file(os.path.join(data_path, 'services_prov_30.geojson'))
district_skateparks_prov = services_prov[services_prov['Administrative'] == adm_name]
#district_skateparks_prov = services_prov[services_prov['Munitipal'] == mun_name]

# Вычисление общего количества скейт-парков
skateparks_count = len(district_skateparks_prov)
print("Общее количество скейт-парков:", skateparks_count)

# Вычисление суммарной вместимости скейт-парков
sum_of_capacity = district_skateparks_prov['capacity'].sum()
print(f"Суммарная вместимость скейт-парков: {sum_of_capacity}")

# Вычисление суммарной вместимости, используемой в границах доступности
carried_capacity_within = district_skateparks_prov['carried_capacity_within'].sum()
print(f"Суммарная вместимость, используемая в границах доступности: {carried_capacity_within}")

# Вычисление суммарной вместимости, используемой вне границ доступности
carried_capacity_without = district_skateparks_prov['carried_capacity_without'].sum()
print(f"Суммарный вместимость, используемая вне границ доступности: {carried_capacity_without}")

# Вычисление суммарной вестимости оставшейся неиспользуемой
capacity_left = district_skateparks_prov['capacity_left'].sum()
print(f"Суммарная вестимость, оставшаяся неиспользуемой: {capacity_left}")

# Вычисление среднего количества посетителей скейт-парка
service_load = round((district_skateparks_prov['service_load'].mean()),0)
print(f"Среднее количество посетителей скейт-парка: {service_load}")

# Вычисление среднего значения загруженности скейт-парков
mean_load = round((district_skateparks_prov['load'].mean()),3)
print(f"Среднее значение загруженности скейт-парков: {mean_load}")

In [ ]:
from iduedu import get_boundary

bounds = get_boundary(osm_id=1115367)

# Создание GeoDataFrame
District = gpd.GeoDataFrame(geometry=[bounds], crs="EPSG:4326")

District['District'] = 'Приморский'
District['total_demand'] = district_build_prov['demand'].sum()
District['total_supplyed_demands_within'] = district_build_prov['supplyed_demands_within'].sum()
District['total_supplyed_demands_without'] = district_build_prov['supplyed_demands_without'].sum()
District['total_demand_left'] = district_build_prov['demand_left'].sum()
District['level_of_demand'] = round(((district_build_prov['demand'].sum() - district_build_prov['demand_left'].sum()) / district_build_prov['demand'].sum()),3)
District['mean_dist'] = round((district_build_prov['avg_dist'].mean()),2)
District['mean_provision'] = round((district_build_prov['provison_value'].mean()),3)

District['skateparks_count'] = len(district_skateparks_prov)
District['total_sum_of_capacity'] = district_skateparks_prov['capacity'].sum()
District['total_carried_capacity_within'] = district_skateparks_prov['carried_capacity_within'].sum()
District['total_carried_capacity_without'] = district_skateparks_prov['carried_capacity_without'].sum()
District['total_capacity_left'] = district_skateparks_prov['capacity_left'].sum()
District['average_service_load'] = round((district_skateparks_prov['service_load'].mean()),0)
District['mean_load_level'] = round((district_skateparks_prov['load'].mean()),3)

District.explore(column='mean_provision', cmap='RdYlGn', tiles='CartoDB positron')